![<caption>](UFPR.PNG)
    
    
IAA007 – Aprendizado de Máquina e Laboratório de IA    
**Prof. Dr. Jaime Wojciechowski**
    
Josué Leal Evangelista

### CLASSIFICAÇÃO


In [1]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)

#### CARREGANDO BIBLIOTECAS

In [2]:
suppressPackageStartupMessages({
library(caret)
library(e1071)
library(data.table)
library(dplyr)
library(ggplot2)
library(tidyr)
})

### 1 - CLASSIFICAÇÃO 

#### 1.1 - CÂNCER DE MAMA

In [3]:
dados_cancer <- fread("Material 02 - 2 - Cancer de Mama - Dados.csv", stringsAsFactors = TRUE)
df_cancer <- as.data.frame(dados_cancer)

In [4]:
head(df_cancer)

Id,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
1,5,1,1,1,2,1,3,1,1,benign
2,5,4,4,5,7,10,3,2,1,benign
3,3,1,1,1,2,2,3,1,1,benign
4,6,8,8,1,3,4,3,7,1,benign
5,4,1,1,3,2,1,3,1,1,benign
6,8,10,10,8,7,10,9,7,1,malignant


In [5]:
df_cancer$Id <- NULL

In [6]:
# VERIFICANDO VALORES NA E NULL
any(is.na(df_cancer))
any(is.null(df_cancer))

[1] FALSE

[1] FALSE

In [7]:
# QUANTIDADE DE OBSERVAÇÕES DE CADA CLASSE
table(df_cancer$Class)


   benign malignant 
      458       241 

In [8]:
set.seed(1971)

In [9]:
# DIVIDINDO DADOS TREINO E TESTE 80% e 20%
indices <- createDataPartition(df_cancer$Class, p = 0.8, list = FALSE)
dados_treino <- df_cancer[indices,]
dados_teste <- df_cancer[-indices,]

### 1.1 - REDES NEURAIS ARTIFICIAIS - RNA

#### APRENDIZADO - MODELO RNA - HOLD-OUT

In [10]:
modelo_rna <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, 
                    dados_treino, 
                    method = "nnet", 
                    trace = FALSE)
modelo_rna

Neural Network 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.9184923  0.7890135
  1     1e-04  0.9409396  0.8619271
  1     1e-01  0.9544217  0.8992793
  3     0e+00  0.9485746  0.8865092
  3     1e-04  0.9427449  0.8731999
  3     1e-01  0.9541973  0.8984119
  5     0e+00  0.9403757  0.8678638
  5     1e-04  0.9467166  0.8822947
  5     1e-01  0.9489740  0.8869525

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 0.1.

In [11]:
previsao_rna <- predict(modelo_rna, dados_teste)

#### MÉTRICAS - MODELO RNA - HOLD-OUT

In [12]:
# MATRIZ DE CONFUSÃO
matriz_confusao1 <- confusionMatrix(previsao_rna, dados_teste$Class)
matriz_confusao1 

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         2
  malignant      1        46
                                          
               Accuracy : 0.9784          
                 95% CI : (0.9382, 0.9955)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.952           
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9583          
         Pos Pred Value : 0.9783          
         Neg Pred Value : 0.9787          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6619          
      Balanced Accuracy : 0.9737          
                                          
       'Positive' Class : benign          

#### APRENDIZADO - MODELO RNA - CROSS VALIDATION

In [13]:
crossvalidation1 <- trainControl(method = "cv", number = 15)
modelo_rna_cv <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, 
                       dados_treino, 
                       trainControl = crossvalidation1, 
                       method = "nnet", 
                       trace = FALSE)
modelo_rna_cv

Neural Network 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.9395720  0.8549139
  1     1e-04  0.9391230  0.8549666
  1     1e-01  0.9561206  0.9019375
  3     0e+00  0.9366515  0.8467950
  3     1e-04  0.9484029  0.8852730
  3     1e-01  0.9523884  0.8934891
  5     0e+00  0.9419730  0.8704081
  5     1e-04  0.9420443  0.8701828
  5     1e-01  0.9509470  0.8899211

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 0.1.

In [14]:
previsao_rna_cv <- predict(modelo_rna_cv, dados_teste)

#### MÉTRICAS - MODELO RNA - CROSS VALIDATION

In [15]:
# MATRIZ DE CONFUSÃO
matriz_confusao2 <- confusionMatrix(previsao_rna_cv, dados_teste$Class)
matriz_confusao2 

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         2
  malignant      1        46
                                          
               Accuracy : 0.9784          
                 95% CI : (0.9382, 0.9955)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.952           
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9583          
         Pos Pred Value : 0.9783          
         Neg Pred Value : 0.9787          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6619          
      Balanced Accuracy : 0.9737          
                                          
       'Positive' Class : benign          

#### APRENDIZADO - MELHOR MODELO RNA - CROSS VALIDATION E TUNEGRID

In [16]:
tunegrid1 <- expand.grid(size = seq(from = 1, to = 10, by = 1), decay = seq(from = 0.1, to = 0.9, by = 0.1))
modelo_rna_best <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, 
                         dados_treino, 
                         method = "nnet", 
                         trainControl = crossvalidation1,
                         tuneGrid = tunegrid1, 
                         MaxNWts = 1000, 
                         maxit = 200, 
                         trace = FALSE)
modelo_rna_best

Neural Network 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
   1    0.1    0.9525230  0.8958234
   1    0.2    0.9538543  0.8984297
   1    0.3    0.9553969  0.9017837
   1    0.4    0.9544163  0.8995636
   1    0.5    0.9540619  0.8987191
   1    0.6    0.9536920  0.8978437
   1    0.7    0.9527122  0.8956529
   1    0.8    0.9531143  0.8964927
   1    0.9    0.9531106  0.8964623
   2    0.1    0.9545681  0.8996931
   2    0.2    0.9553958  0.9014842
   2    0.3    0.9559455  0.9026315
   2    0.4    0.9553603  0.9014221
   2    0.5    0.9548069  0.9001455
   2    0.6    0.9546371  0.8997209
   2    0.7    0.9550169  0.9006745
   2    0.8    0.9548115  0.9001343
   2    0.9    0.9542484  0.8988474
   3    0.1    0.9515230  0.8927649
   3    0.2    0.9555861  0.9020211
   3

In [17]:
previsao_rna_best <- predict(modelo_rna_best, dados_teste)

#### MÉTRICAS - MELHOR MODELO RNA - CROSS VALIDATION E TUNEGRID

In [18]:
# MATRIZ DE CONFUSÃO
matriz_confusao3 <- confusionMatrix(previsao_rna_best, dados_teste$Class)
matriz_confusao3

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         2
  malignant      1        46
                                          
               Accuracy : 0.9784          
                 95% CI : (0.9382, 0.9955)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.952           
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9583          
         Pos Pred Value : 0.9783          
         Neg Pred Value : 0.9787          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6619          
      Balanced Accuracy : 0.9737          
                                          
       'Positive' Class : benign          

### 1.2 - K-NEAREST NEIGHBOR - KNN

#### APRENDIZADO - MODELO KNN - TUNEGRID

In [19]:
tunegrid2 <- expand.grid(k = c(3, 5, 7, 9))

In [20]:
modelo_knn <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, 
                    dados_treino, 
                    method = "knn", 
                    tuneGrid = tunegrid2)
modelo_knn

k-Nearest Neighbors 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa    
  3  0.9405889  0.8672248
  5  0.9471748  0.8821841
  7  0.9481063  0.8847846
  9  0.9510715  0.8913537

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 9.

In [21]:
previsao_knn <-predict(modelo_knn, dados_teste)

#### MÉTRICAS - MODELO KNN - TUNEGRID

In [22]:
# MATRIZ DE CONFUSÃO
matriz_confusao4 <- confusionMatrix(previsao_knn, dados_teste$Class)
matriz_confusao4

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        89         2
  malignant      2        46
                                         
               Accuracy : 0.9712         
                 95% CI : (0.928, 0.9921)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9364         
                                         
 Mcnemar's Test P-Value : 1              
                                         
            Sensitivity : 0.9780         
            Specificity : 0.9583         
         Pos Pred Value : 0.9780         
         Neg Pred Value : 0.9583         
             Prevalence : 0.6547         
         Detection Rate : 0.6403         
   Detection Prevalence : 0.6547         
      Balanced Accuracy : 0.9682         
                                         
       'Positive' Class : benign         
                   

### 1.3 - SUPPORT VECTOR MACHINES - SVM

In [23]:
install.packages("kernlab")


  There is a binary version available but the source version is later:
        binary source needs_compilation
kernlab 0.9-29 0.9-30              TRUE

  Binaries will be installed
package 'kernlab' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lealj\AppData\Local\Temp\RtmpeYKZzR\downloaded_packages


#### APRENDIZADO - MODELO SVM - HOLD-OUT

In [24]:
modelo_svm <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, 
                    dados_treino, 
                    method = "svmRadial")
modelo_svm

Support Vector Machines with Radial Basis Function Kernel 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.9449998  0.8813937
  0.50  0.9504537  0.8925212
  1.00  0.9505158  0.8919770

Tuning parameter 'sigma' was held constant at a value of 3.05936
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 3.05936 and C = 1.

In [25]:
previsao_svm <- predict(modelo_svm, dados_teste)

#### MÉTRICAS - MODELO SVM - HOLD-OUT

In [26]:
# MATRIZ DE CONFUSÃO
matriz_confusao5 <- confusionMatrix(previsao_svm, dados_teste$Class)
matriz_confusao5

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         1
  malignant      4        47
                                          
               Accuracy : 0.964           
                 95% CI : (0.9181, 0.9882)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9216          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.9560          
            Specificity : 0.9792          
         Pos Pred Value : 0.9886          
         Neg Pred Value : 0.9216          
             Prevalence : 0.6547          
         Detection Rate : 0.6259          
   Detection Prevalence : 0.6331          
      Balanced Accuracy : 0.9676          
                                          
       'Positive' Class : benign          

#### APRENDIZADO - MODELO SVM - CROSS VALIDATION

In [27]:
crossvalidation2 <- trainControl(method = "cv", number = 15)
modelo_svm_cv <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin,
                       dados_treino, 
                       trainControl = crossvalidation2, 
                       method = "svmRadial")
modelo_svm_cv

Support Vector Machines with Radial Basis Function Kernel 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.9515435  0.8954519
  0.50  0.9541904  0.9007284
  1.00  0.9533551  0.8985114

Tuning parameter 'sigma' was held constant at a value of 1.86829
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 1.86829 and C = 0.5.

In [28]:
previsao_svm_cv <- predict(modelo_svm_cv, dados_teste)

#### MÉTRICAS - MODELO SVM - CROSS VALIDATION

In [29]:
# MATRIZ DE CONFUSÃO
matriz_confusao6 <- confusionMatrix(previsao_svm_cv, dados_teste$Class)
matriz_confusao6

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         1
  malignant      4        47
                                          
               Accuracy : 0.964           
                 95% CI : (0.9181, 0.9882)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9216          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.9560          
            Specificity : 0.9792          
         Pos Pred Value : 0.9886          
         Neg Pred Value : 0.9216          
             Prevalence : 0.6547          
         Detection Rate : 0.6259          
   Detection Prevalence : 0.6331          
      Balanced Accuracy : 0.9676          
                                          
       'Positive' Class : benign          

#### APRENDIZADO - MODELO SVM - CROSS VALIDATION E TUNEGRID

In [30]:
tunegrid2 <- expand.grid(C = c(1, 2, 10, 50, 100), sigma = c(0.01, 0.015, 0.2))
modelo_svm_best <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin,
                         dados_treino, 
                         method = "svmRadial", 
                         trainControl = crossvalidation2,
                         tuneGrid = tunegrid2)
modelo_svm_best

Support Vector Machines with Radial Basis Function Kernel 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  C    sigma  Accuracy   Kappa    
    1  0.010  0.9554408  0.9014353
    1  0.015  0.9558161  0.9023677
    1  0.200  0.9559033  0.9034702
    2  0.010  0.9556229  0.9019317
    2  0.015  0.9554313  0.9016322
    2  0.200  0.9533770  0.8981097
   10  0.010  0.9570154  0.9053101
   10  0.015  0.9574277  0.9063823
   10  0.200  0.9424702  0.8731297
   50  0.010  0.9578205  0.9073420
   50  0.015  0.9570650  0.9058299
   50  0.200  0.9276681  0.8387517
  100  0.010  0.9572502  0.9062388
  100  0.015  0.9565151  0.9047241
  100  0.200  0.9234129  0.8286700

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.01 and C = 50.

In [31]:
previsao_svm_best <- predict(modelo_svm_best, dados_teste)

#### MÉTRICAS - MODELO SVM - CROSS VALIDATION E TUNEGRID

In [32]:
# MATRIZ DE CONFUSÃO
matriz_confusao7 <- confusionMatrix(previsao_svm_best, dados_teste$Class)
matriz_confusao7

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         2
  malignant      1        46
                                          
               Accuracy : 0.9784          
                 95% CI : (0.9382, 0.9955)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.952           
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9583          
         Pos Pred Value : 0.9783          
         Neg Pred Value : 0.9787          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6619          
      Balanced Accuracy : 0.9737          
                                          
       'Positive' Class : benign          

### 1.4 - RANDOM FOREST 

#### APRENDIZADO - MODELO RANDOM FOREST  - HOLD-OUT

In [33]:
modelo_rf <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, dados_treino, method = "rf")
modelo_rf

Random Forest 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.9548322  0.9007702
  3     0.9526771  0.8960362
  5     0.9480447  0.8858049

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [34]:
previsao_rf <- predict(modelo_rf, dados_teste)

#### MÉTRICAS - MODELO RANDOM FOREST  - HOLD-OUT

In [35]:
# MATRIZ DE CONFUSÃO
matriz_confusao8 <- confusionMatrix(previsao_rf, dados_teste$Class)
matriz_confusao8 

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         2
  malignant      1        46
                                          
               Accuracy : 0.9784          
                 95% CI : (0.9382, 0.9955)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.952           
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9583          
         Pos Pred Value : 0.9783          
         Neg Pred Value : 0.9787          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6619          
      Balanced Accuracy : 0.9737          
                                          
       'Positive' Class : benign          

#### APRENDIZADO - MODELO RANDOM FOREST  - CROSS VALIDATION

In [36]:
crossvalidation3 <- trainControl(method = "cv", number = 8)
modelo_rf_cv <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin, 
                      dados_treino, 
                      method = "rf",
                      trainControl = crossvalidation3)
modelo_rf_cv

Random Forest 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.9460928  0.8818979
  3     0.9411061  0.8707241
  5     0.9369463  0.8613990

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [37]:
previsao_rf_cv <- predict(modelo_rf_cv, dados_teste)

#### MÉTRICAS - MODELO RANDOM FOREST  - CROSS VALIDATION

In [38]:
# MATRIZ DE CONFUSÃO
matriz_confusao9 <- confusionMatrix(previsao_rf_cv, dados_teste$Class)
matriz_confusao9

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        89         2
  malignant      2        46
                                         
               Accuracy : 0.9712         
                 95% CI : (0.928, 0.9921)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9364         
                                         
 Mcnemar's Test P-Value : 1              
                                         
            Sensitivity : 0.9780         
            Specificity : 0.9583         
         Pos Pred Value : 0.9780         
         Neg Pred Value : 0.9583         
             Prevalence : 0.6547         
         Detection Rate : 0.6403         
   Detection Prevalence : 0.6547         
      Balanced Accuracy : 0.9682         
                                         
       'Positive' Class : benign         
                   

#### APRENDIZADO - MODELO RANDOM FOREST  - CROSS VALIDATION E TUNEGRID

In [39]:
tunegrid3 <- expand.grid(mtry = c(2, 4, 7, 9))
modelo_rf_best <- train(Class ~ Cell.size + Cell.shape + Bare.nuclei + Normal.nucleoli + Bl.cromatin,
                        dados_treino, 
                        method = "rf", 
                        trainControl = crossvalidation3,
                        tuneGrid = tunegrid3)
modelo_rf_best

Random Forest 

560 samples
  5 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.9517543  0.8949959
  4     0.9449963  0.8801236
  7     0.9411029  0.8714630
  9     0.9432293  0.8762325

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [40]:
previsao_rf_best <- predict(modelo_rf_best, dados_teste)

#### MÉTRICAS - MODELO RANDOM FOREST  - CROSS VALIDATION E TUNEGRID

In [41]:
# MATRIZ DE CONFUSÃO
matriz_confusao10 <- confusionMatrix(previsao_rf_best, dados_teste$Class)
matriz_confusao10

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         2
  malignant      1        46
                                          
               Accuracy : 0.9784          
                 95% CI : (0.9382, 0.9955)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.952           
                                          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9583          
         Pos Pred Value : 0.9783          
         Neg Pred Value : 0.9787          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6619          
      Balanced Accuracy : 0.9737          
                                          
       'Positive' Class : benign          